In [1]:
from gensim.models.word2vec import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

In [2]:
import warnings
warnings.filterwarnings("ignore")
from collections import defaultdict
from tqdm.autonotebook import tqdm
#import pandas as pd
import mysql.connector
import codecs
codecs.register(lambda name: codecs.lookup('utf8') if name == 'utf8mb4' else None)
import conllu

In [3]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd='password',
  #database="small_commits_db"
  database='commits_db',
  charset='utf8mb4')

mycursor = mydb.cursor()

## Data by time

In [62]:
def yield_lemma(year, month, mode='exact'):
    dt = str(year)+'-'+month
    if mode == 'exact':
        mycursor.execute('SELECT conllu FROM C WHERE date LIKE "{}%"'.format(dt))
    elif mode == 'before':
        mycursor.execute('SELECT conllu FROM C WHERE date < %s', (str(year),))
    else:
        mycursor.execute('SELECT conllu FROM C WHERE date >= %s', (str(year),))
    text = mycursor.fetchone()
    #print (text)
    while text:
        doc = conllu.parse(text[0])
        for sent in doc:
            snt = []
            for i in sent.tokens:
                snt.append(i['lemma'])
            yield snt
        text = mycursor.fetchone()

In [66]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd='password',
  #database="small_commits_db"
  database='commits_db',
  charset='utf8mb4')

mycursor = mydb.cursor()

In [67]:
with open('../data/by_month/1999-before.txt', 'w') as file:
    for item in tqdm(yield_lemma(2000, '00', mode='before')):
        sent = item
        print(' '.join(sent).strip(), file=file)

In [68]:
with open('../data/by_month/2019-after.txt', 'w') as file:
    for item in tqdm(yield_lemma(2019, '00', mode='after')):
        sent = item
        print(' '.join(sent).strip(), file=file)

In [89]:
a = 2000
b = 2019
x = ['01', '02', '03', '04','05','06', '07','08','09', '10','11','12']

In [90]:
for year in tqdm(range(a, b)):
    for month in tqdm(x, leave=False):
        with open('../data/by_month/{}-{}.txt'.format(str(year), month), 'w') as file:
            for item in tqdm(yield_lemma(year, month, mode='exact'), leave=False):
                sent = item
                print(' '.join(sent).strip(), file=file)

## Frequency dict

In [3]:
! wc -l frequency_dict_no_pos_proc.csv

113983 frequency_dict_no_pos_proc.csv


In [4]:
! tail -n 5 frequency_dict_no_pos_proc.csv

decisiontable	0	0	0	0	0	0	101	0	0	0	0	0	0	0	0	0	101
19955	12	5	15	0	2	1	1	2	0	31	5	7	0	0	3	17	101
webtextrenderer	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	101	101
37051	4	0	74	0	2	0	1	0	0	7	1	7	1	0	0	4	101
rmlock	100	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	101


In [5]:
freq_dict = {}
k = 0
for line in open('frequency_dict_no_pos_proc.csv'):
    if k == 0:
        k += 1
    else:
        row = line.strip().split('\t')
        if row[0].isalpha() and int(row[-1]) > 500:
            freq_dict[row[0]] = int(row[-1])
print(len(freq_dict))

25488


## W2V time

In [11]:
w2v = Word2Vec(workers=8, size=300, seed=23)

In [12]:
w2v.build_vocab_from_freq(freq_dict, update=False)

In [13]:
import os
from collections import defaultdict
lsr = os.listdir('../data/by_month/')
yrs = defaultdict(list)
for i in lsr:
    yrs[i[:4]].append('../data/by_month/'+i)
yrs = dict(yrs)

In [14]:
for month in (sorted(yrs['2000'])):
    print(month)

../data/by_month/2000-01.txt
../data/by_month/2000-02.txt
../data/by_month/2000-03.txt
../data/by_month/2000-04.txt
../data/by_month/2000-05.txt
../data/by_month/2000-06.txt
../data/by_month/2000-07.txt
../data/by_month/2000-08.txt
../data/by_month/2000-09.txt
../data/by_month/2000-10.txt
../data/by_month/2000-11.txt
../data/by_month/2000-12.txt


In [15]:
for year in tqdm(sorted(yrs)):
    #fname = '../data/by_month_vecs/'+str(year)+'.txt'
    for month in tqdm(sorted(yrs[year]), leave=False):
        fname = month.replace('by_month', 'by_month_vecs')
        k = 0
        w = 0
        for line in tqdm(open(month), leave=False):
            w += len(line.split())
            k += 1
        w2v.train(total_examples=k, total_words = w, corpus_file=month, epochs=10)
        w2v.wv.save_word2vec_format(fname, binary=False)